In [19]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdmolops
import numpy as np
from scipy.sparse import block_diag
from spektral.data import Graph, Dataset, Loader, DisjointLoader
from spektral.data import SingleLoader
from spektral.layers import GCNConv, GlobalSumPool
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
import tensorflow as tf

## Feature engineering and model generation

In [40]:
df = pd.read_csv('../data/train_subset.tsv').head(10000)
df

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,0,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,BRD4,0
1,1,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,HSA,0
2,2,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,sEH,0
3,3,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.NCc1cccc(Br)n1,C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...,BRD4,0
4,4,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.NCc1cccc(Br)n1,C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...,HSA,0
...,...,...,...,...,...,...,...
9995,9995,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C=C(C)COCCN.Cl,NCCNC(=O)c1cccnc1,C#CC[C@@H](CC(=O)N[Dy])Nc1nc(NCCNC(=O)c2cccnc2...,sEH,0
9996,9996,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C=C(C)COCCN.Cl,NCCOc1cccnc1,C#CC[C@@H](CC(=O)N[Dy])Nc1nc(NCCOCC(=C)C)nc(NC...,BRD4,0
9997,9997,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C=C(C)COCCN.Cl,NCCOc1cccnc1,C#CC[C@@H](CC(=O)N[Dy])Nc1nc(NCCOCC(=C)C)nc(NC...,HSA,0
9998,9998,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C=C(C)COCCN.Cl,NCCOc1cccnc1,C#CC[C@@H](CC(=O)N[Dy])Nc1nc(NCCOCC(=C)C)nc(NC...,sEH,0


In [41]:
# Convert SMILES to graph
from spektral.data import Graph

def mol_to_graph(mol):
    atoms = mol.GetAtoms()
    edges = [(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()) for bond in mol.GetBonds()]
    a = rdmolops.GetAdjacencyMatrix(mol)
    x = np.array([atom.GetAtomicNum() for atom in atoms]).reshape(-1, 1).astype(np.float32)  # Convert to float32 here
    return Graph(x=x, a=a)

# Convert SMILES to Graph
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol_to_graph(mol) if mol else None

def pad_features(features, max_features):
    padded_length = max_features - features.shape[1]
    padded_features = np.pad(features, ((0, 0), (0, padded_length)), 'constant', constant_values=0)
    return padded_features.astype(np.float32)  # Ensure the padded array is float32

def process_df(df):
    smile_columns = ['buildingblock1_smiles', 'buildingblock2_smiles', 'buildingblock3_smiles', 'molecule_smiles']
    unified_graph_list = []
    for _, row in df.iterrows():
        graphs = [smiles_to_graph(row[col]) for col in smile_columns]
        graphs = [graph for graph in graphs if graph is not None]
        if len(graphs) == 4:
            max_features = max(graph.x.shape[1] for graph in graphs)
            combined_x = np.vstack([pad_features(graph.x, max_features) for graph in graphs])
            combined_a = block_diag([graph.a for graph in graphs]).tocoo()  # Keep adjacency matrix in COO format
            label = row['binds']  # Extract the label
            unified_graph_list.append(Graph(x=combined_x, a=combined_a, y=label))
    return unified_graph_list




In [65]:
import numpy as np
from scipy.sparse import csr_matrix
from spektral.data import Graph, Dataset
from tensorflow.keras.utils import to_categorical


class MoleculeDataset(Dataset):
    def __init__(self, graph_list, **kwargs):
        self.graph_list = graph_list
        super().__init__(**kwargs)
    
    def read(self):
        return self.graph_list

    def collate(batch):
        features = [item.x for item in batch]
        adj_matrices = [item.a for item in batch]
        labels = [item.y for item in batch]  # Assuming labels are included in the graph objects
    
        features = np.vstack(features)
        adj_matrices = [adj.tocoo() for adj in adj_matrices]  # Convert to COO format if not already
        adj_indices = [np.column_stack((adj.row, adj.col)) for adj in adj_matrices]
        adj_values = [adj.data for adj in adj_matrices]
        adj_shape = adj_matrices[0].shape if adj_matrices else (0, 0)
    
        # Convert to TensorFlow SparseTensor
        adj_matrices = [tf.SparseTensor(indices, values, adj_shape) for indices, values in zip(adj_indices, adj_values)]
        adj_matrices = tf.sparse.concat(axis=0, sp_inputs=adj_matrices)
    
        labels = np.array(labels)  # Ensure labels are a numpy array
    
        return [features, adj_matrices], labels
        
graphs = process_df(df)

In [52]:
import spektral
import tensorflow as tf
from spektral.layers import GraphSageConv, GlobalAvgPool
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense



class GNN(tf.keras.Model):
    def __init__(self, activation='relu'):
        super(GNN, self).__init__()
        self.graph_conv1 = GraphSageConv(32, activation=activation)
        self.graph_conv2 = GraphSageConv(32, activation=activation)
        self.pool = GlobalAvgPool()
        self.classifier = Dense(1, activation='sigmoid')

    def call(self, inputs):
        x, a, i = inputs
        # Pass only x and a to the GraphSageConv layers
        x = self.graph_conv1([x, a])
        x = self.graph_conv2([x, a])
        # Use i in the global pooling layer if necessary
        x = self.pool([x, i])
        return self.classifier(x)

model = GNN()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [87]:
from scipy.sparse import coo_matrix, save_npz
import numpy as np
import pickle
    
def save_graphs_pickle(graphs, filename):
    with open(filename, 'wb') as f:
        pickle.dump(graphs, f, protocol=pickle.HIGHEST_PROTOCOL)

def load_graphs_pickle(filename):
    with open(filename, 'rb') as f:
        graphs = pickle.load(f)
    return graphs

save_graphs_pickle(graphs, '/Users/admin/kaggle/belka/data/belka_graph_data.pickle')

In [88]:
graphs = load_graphs_pickle('/Users/admin/kaggle/belka/data/belka_graph_data.pickle')
train_graphs, val_graphs = train_test_split(graphs, test_size=0.2, random_state=42)
train_dataset = MoleculeDataset(train_graphs)
val_dataset = MoleculeDataset(val_graphs)
train_loader = DisjointLoader(train_dataset, node_level=True, batch_size=32, shuffle=False)
val_loader = DisjointLoader(val_dataset, node_level=True, batch_size=32, shuffle=False)

In [89]:
# Assuming 'loader' is your DisjointLoader instance
# model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=10, verbose=1)
model.fit(train_loader.load(),
          validation_data=val_loader.load(),
          steps_per_epoch=train_loader.steps_per_epoch,
          validation_steps=val_loader.steps_per_epoch,
          epochs=10,
          verbose=1)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9989 - loss: 0.0088 - val_accuracy: 0.9985 - val_loss: 0.0115
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9989 - loss: 0.0088 - val_accuracy: 0.9985 - val_loss: 0.0115
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9989 - loss: 0.0088 - val_accuracy: 0.9985 - val_loss: 0.0115
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9989 - loss: 0.0088 - val_accuracy: 0.9985 - val_loss: 0.0115
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9989 - loss: 0.0088 - val_accuracy: 0.9985 - val_loss: 0.0115
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9989 - loss: 0.0088 - val_accuracy: 0.9985 - val_loss: 0.0115
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9989 - loss: 0.0088 - val_accuracy: 0.9985 - val_loss: 0.0115
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9989 - loss: 0.0088 - val_accuracy: 0.

### Tune Hyperparameters

In [93]:
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from sklearn.model_selection import train_test_split

def train_gnn(config, checkpoint_dir=None):
    from spektral.data import DisjointLoader
    
    graphs = load_graphs_pickle('/Users/admin/kaggle/belka/data/belka_graph_data.pickle')
    train_graphs, val_graphs = train_test_split(graphs, test_size=0.2, random_state=42)
    train_dataset = MoleculeDataset(train_graphs)
    val_dataset = MoleculeDataset(val_graphs)
    train_loader = DisjointLoader(train_dataset, node_level=True, batch_size=32, shuffle=False)
    val_loader = DisjointLoader(val_dataset, node_level=True, batch_size=32, shuffle=False)

    model = GNN(activation=config['activation'])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    callback = tf.keras.callbacks.TensorBoard(log_dir='./logs')
    history = model.fit(train_loader.load(),
                        validation_data=val_loader.load(),
                        epochs=config['num_epochs'],
                        callbacks=[callback],
                        steps_per_epoch=train_loader.steps_per_epoch,
                        validation_steps=val_loader.steps_per_epoch,
                        verbose=1)

    loss = history.history['loss'][-1]
    train.report({'loss': loss})


scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=50,
    grace_period=10,
    reduction_factor=2
)

config = {
    "lr": tune.loguniform(1e-4, 1e-1),
    "dropout_rate": tune.uniform(0.1, 0.5),
    "activation": tune.choice(['relu', 'tanh', 'sigmoid', 'leaky_relu']),
    "num_epochs": tune.choice([10, 20, 30])
}

result = tune.run(
    tune.with_parameters(train_gnn),
    config=config,
    num_samples=10,
    resources_per_trial={"cpu": 1, "gpu": 0},
    scheduler=scheduler,
    progress_reporter=tune.CLIReporter(metric_columns=["loss", "training_iteration"])
)

best_trial = result.get_best_trial("loss", "min", "last")
print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))

2024-05-01 11:32:04,459	INFO tune.py:624 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-05-01 11:32:04 (running for 00:00:00.18)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 40.000: None | Iter 20.000: None | Iter 10.000: None
Logical resource usage: 10.0/14 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-05-01_10-41-20_376415_1121/artifacts/2024-05-01_11-32-04/train_gnn_2024-05-01_11-32-04/driver_artifacts
Number of trials: 10/10 (10 PENDING)


== Status ==
Current time: 2024-05-01 11:32:09 (running for 00:00:05.24)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 40.000: None | Iter 20.000: None | Iter 10.000: None
Logical resource usage: 10.0/14 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-05-01_10-41-20_376415_1121/artifacts/2024-05-01_11-32-04/train_gnn_2024-05-01_11-32-04/driver_artifacts
Number of trials: 10/10 (10 RUNNING)


== Status ==
Current time: 2024-05-01 11:32:14 (running for 00:00:10.34)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 40.000: None | Iter 20.000: None | Iter 10.000: None
Logical resource u

2024-05-01 11:55:36,915	WARNING tune.py:229 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-05-01 11:55:36,926	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-05-01 11:55:36,930	INFO tune.py:1021 -- Wrote the latest version of all result files and experiment state to '/Users/admin/ray_results/train_gnn_2024-05-01_11-32-04' in 0.0

== Status ==
Current time: 2024-05-01 11:55:36 (running for 00:23:32.45)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 40.000: None | Iter 20.000: None | Iter 10.000: None
Logical resource usage: 10.0/14 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-05-01_10-41-20_376415_1121/artifacts/2024-05-01_11-32-04/train_gnn_2024-05-01_11-32-04/driver_artifacts
Number of trials: 10/10 (10 RUNNING)
+-----------------------+----------+----------------+--------------+----------------+-------------+--------------+
| Trial name            | status   | loc            | activation   |   dropout_rate |          lr |   num_epochs |
|-----------------------+----------+----------------+--------------+----------------+-------------+--------------|
| train_gnn_1bab6_00000 | RUNNING  | 127.0.0.1:6455 | tanh         |       0.489718 | 0.000126517 |           20 |
| train_gnn_1bab6_00001 | RUNNING  | 127.0.0.1:6456 | leaky_relu   |       0.241048 | 0.00993345  |           20 |
| train_gnn_1bab6_00002 

2024-05-01 11:55:47,001	INFO tune.py:1053 -- Total run time: 1422.54 seconds (1412.44 seconds for the tuning loop).
2024-05-01 11:55:47,002	WARNING tune.py:1068 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)
2024-05-01 11:55:47,006	WARNING experiment_analysis.py:568 -- Could not find best trial. Did you pass the correct `metric` parameter?


AttributeError: 'NoneType' object has no attribute 'config'